In [1]:
import json

# Open the dataset JSON
with open("dataSets/ingredient_and_instructions.json", "r") as dataSet:
    data = json.load(dataSet)

recipie = "" # The individual recipie
recipie_list = [] # Where we will store the recipie for training

for meal in data:
    # Get the recipie name and its instructions
    meal_info = data[meal]


    # Get the ingredients
    ingredients = meal_info["ingredient_sections"]

    for i in ingredients:
        # Retrieve ingredients and their details
        item = ""           # Dough
        qty = ""            # ½
        unit = ""           # cup
        ingredient = ""     # unsalted butter
        primary_unit = { "quantity": None, "display": None }

        # Some ingredients arent named, but are still listed as "1 cup" or similar
        # Additionally, if there is an ingredient name, it may already have a colon
        # in front of it, so we need to check for that before formatting
        if i["name"]:
            if i["name"][-1] != ":": # If there isnt a colon, add one
                item = i["name"] + ": \n"
            else: # Otherwise, just add the name
                item = i["name"] + " \n"
        else:
            item = ""

        recipie += item # Add the ingredient name to the recipie

        # Loop over each ingredient
        j = 0
        while j < len(i["ingredients"]):
            # i["ingredients"][0] may not exist, use try/except to prevent
            # any ListIndexErrors from accessing an index that doesn't exist
            try: primary_unit = i["ingredients"][j]["primary_unit"]
            except: primary_unit = { "quantity": None, "display": None } # Dummy object

            # Some recipies don't list an ingredient at all
            try: ingredient += i["ingredients"][j]["name"]
            except: ingredient += ""

            # If the try/except blocks above are successful, we can set the values
            # if primary_unit["quantity"]: qty = primary_unit["quantity"] + " "
            # if primary_unit["display"]: unit = primary_unit["display"] + " "

            # Construct the ingredient string
            recipie += qty + unit  + ingredient + "\n"
            ingredient = ""
            j += 1

        recipie += "\n"

    # Newline before adding instructions
    recipie += "\n"
    recipie += meal.replace("-", " ").title() + "\n\n"
    recipe_instructions = meal_info["instructions"]

    # Loop over the instructions and format them
    step = 1
    for text in recipe_instructions:
        instruction = text["display_text"]
        recipie += str(step) + ". " + instruction + "\n"
        step += 1

    # Add the recipie to the list
    recipie_list.append(recipie)

    # Free memory for the next recipie
    recipie = ""
    recipe_instructions = []
    ingredients = []
    item = ""
    qty = ""
    unit = ""
    ingredient = ""

# For testing purposes, we can output a random recipie
import random
print(recipie_list[random.randint(0, len(recipie_list) - 1)])

#print(recipie_list[0])


double cream
cream cheese
icing sugar
vanilla ice cream
whole milk
red velvet cake
red gel food coloring


Red Velvet Cake Milkshake

1. To make the cream cheese topping, whip the double cream, cream cheese, and icing sugar together. Set aside.
2. In a blender, combine ice cream, whole milk, 2 slices of red velvet cake, and red food gel color and blend until smooth.
3. Pour into a glass and top with the cream cheese, another slice of red velvet cake, and a straw.
4. ENJOY!



In [2]:
#CSV File 1 Record per row
#Record: 1 string with ingredients + recipe


# [[recipe1], [recipe2], [recipe3]]

import csv

with open("recipe_fixed.csv", 'w', encoding="utf-8") as csvfile:
    csvwriter = csv.writer(csvfile)
    for recipie in recipie_list:
        csvwriter.writerow([recipie])


--- MODEL TRANING ---

In [3]:
#check if there is a gpu available
import torch
print(torch.cuda.is_available())

True


In [6]:
from aitextgen.TokenDataset import TokenDataset
from aitextgen.tokenizers import train_tokenizer
from aitextgen.utils import GPT2ConfigCPU
from aitextgen import aitextgen
import numpy

tokenizer_file = "aitextgen.tokenizer.json"
config = GPT2ConfigCPU()
ai = aitextgen(tokenizer_file=tokenizer_file, to_gpu=True)

In [8]:
#Ai training disabled to prevent accidental overrides.

file_name = "recipe_fixed.csv"
train_tokenizer(file_name)
data = TokenDataset(file_name, tokenizer_file=tokenizer_file, block_size=64)
ai.train(data, batch_size=100, num_steps=64000, generate_every=500, save_every=2000)
ai.generate(prompt="potato, butter, salt, oregeno, cheese, sour cream")

You are tokenizing a CSV file, but you did not set line_by_line=True. Please change if unintended.


  0%|          | 0/9922 [00:00<?, ?it/s]

pytorch_model.bin already exists in /trained_model and will be overwritten!
Windows does not support multi-GPU training. Setting to 1 GPU.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


  0%|          | 0/64000 [00:00<?, ?it/s]

500 steps reached: generating sample texts.
re in the refrigerator for at least 1 hour, or up to 4 hours.
2. In a small saucepan, bring glaze ingredients to a low boil over medium heat. Simmer for 3 minutes, stirring frequently.
3. Heat mixture and 2 cups (4 tablespoon of olive oil until a large woking time).
4. Add in a ½ cupcake lasagula. Smedium carrot.
4. Spoon soft ball, and ½ cupcake panemash the meatbeeze your hands out meat. Spoon hashing cups out onto acult sunce the meatbab up the blend the meatbase hole for the meat, then adding cup. ½ cup, then poft edge into the potato cups inside casting cup. Dar, and cir hole sn keek into the snuss into the snonstick cooking liquick. Then the meat, mipped snight ott shornight down the knudges and scoo mushipped soaked sm
1,000 steps reached: generating sample texts.
 Corn Dogs

1. In a medium bowl, mix together McCormick Cumin, garlic powder, paprika, salt, and pepper.
2. Spread ricotta on porking belly, making sure not to evenly cover t

In [12]:
# Load the trained model
ai = aitextgen(model_folder="trained_model", tokenizer_file="aitextgen.tokenizer.json", to_gpu=True, block_size="64")
ai.generate(temperature=.7, prompt="steak")